# Benin Solar Data - Exploratory Data Analysis

**Dataset:** benin-malanville.csv  
**Objective:** Profile, clean, and explore the Benin solar dataset to identify key trends and insights for solar investment decisions.

## Table of Contents
1. Setup & Data Loading
2. Summary Statistics & Missing Values
3. Outlier Detection & Data Quality
4. Data Cleaning
5. Time Series Analysis
6. Cleaning Impact Analysis
7. Correlation & Relationship Analysis
8. Wind Analysis & Distributions
9. Temperature & Humidity Analysis
10. Bubble Chart Analysis
11. Key Insights & Conclusions


## 1. Setup & Data Loading


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Figure size defaults
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")


Libraries imported successfully!


In [ ]:
# Load the dataset
data_path = '../data/benin-malanville.csv'
df = pd.read_csv(data_path)

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")


In [ ]:
# Initial data inspection
print("=" * 80)
print("FIRST 5 ROWS:")
print("=" * 80)
display(df.head())

print("\n" + "=" * 80)
print("LAST 5 ROWS:")
print("=" * 80)
display(df.tail())

print("\n" + "=" * 80)
print("DATA TYPES:")
print("=" * 80)
display(df.dtypes)


## 2. Summary Statistics & Missing Values


In [ ]:
# Summary statistics for all numeric columns
print("=" * 80)
print("SUMMARY STATISTICS FOR NUMERIC COLUMNS")
print("=" * 80)
summary_stats = df.describe()
display(summary_stats)


In [ ]:
# Missing values analysis
print("=" * 80)
print("MISSING VALUES ANALYSIS")
print("=" * 80)

missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing Count': missing_values.values,
    'Missing Percentage': missing_percentage.values
})

missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Percentage', ascending=False)

if len(missing_df) > 0:
    display(missing_df)
    
    # Flag columns with >5% missing values
    high_missing = missing_df[missing_df['Missing Percentage'] > 5]
    if len(high_missing) > 0:
        print("\n⚠️ Columns with >5% missing values:")
        for col in high_missing['Column']:
            print(f"  - {col}: {high_missing[high_missing['Column']==col]['Missing Percentage'].values[0]:.2f}%")
else:
    print("✓ No missing values found in the dataset!")


## 3. Outlier Detection & Data Quality


In [ ]:
# Check for negative values in solar radiation columns
print("=" * 80)
print("DATA QUALITY CHECK: NEGATIVE VALUES IN SOLAR RADIATION")
print("=" * 80)

solar_cols = ['GHI', 'DNI', 'DHI']
for col in solar_cols:
    if col in df.columns:
        negative_count = (df[col] < 0).sum()
        negative_pct = (negative_count / len(df)) * 100
        print(f"{col}: {negative_count} negative values ({negative_pct:.2f}%)")
        
print("\nNote: Negative values during nighttime are expected, but should be investigated for daytime readings.")


In [ ]:
# Compute Z-scores for outlier detection
print("=" * 80)
print("OUTLIER DETECTION USING Z-SCORES (|Z| > 3)")
print("=" * 80)

# Columns to check for outliers
outlier_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# Create a copy for outlier analysis
df_outlier = df.copy()

# Store outlier information
outlier_summary = []

for col in outlier_cols:
    if col in df.columns:
        # Calculate Z-scores
        z_scores = np.abs(stats.zscore(df[col].dropna()))
        
        # Count outliers (|Z| > 3)
        outliers = np.sum(z_scores > 3)
        outlier_pct = (outliers / len(df[col].dropna())) * 100
        
        outlier_summary.append({
            'Column': col,
            'Outliers': outliers,
            'Outlier Percentage': outlier_pct
        })

outlier_df = pd.DataFrame(outlier_summary)
display(outlier_df)


## 4. Data Cleaning


In [ ]:
# Create a cleaned copy of the dataset
df_clean = df.copy()

print("=" * 80)
print("DATA CLEANING PROCESS")
print("=" * 80)

print(f"\nOriginal dataset shape: {df_clean.shape}")

# Step 1: Handle missing values in key columns with median imputation
key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb', 'RH', 'WS', 'WSgust', 'WD', 'BP']

for col in key_columns:
    if col in df_clean.columns:
        missing_before = df_clean[col].isnull().sum()
        if missing_before > 0:
            median_val = df_clean[col].median()
            df_clean[col].fillna(median_val, inplace=True)
            print(f"✓ {col}: Imputed {missing_before} missing values with median ({median_val:.2f})")

# Step 2: Remove rows with critical missing data (Comments can be missing)
# Only drop rows where all key sensor readings are missing
critical_cols = ['GHI', 'DNI', 'DHI']
rows_before = len(df_clean)
df_clean = df_clean.dropna(subset=critical_cols, how='all')
rows_after = len(df_clean)
rows_dropped = rows_before - rows_after

if rows_dropped > 0:
    print(f"\n✓ Dropped {rows_dropped} rows with all critical solar radiation values missing")

print(f"\nCleaned dataset shape: {df_clean.shape}")
print(f"Rows removed: {len(df) - len(df_clean)}")
print(f"Remaining missing values: {df_clean.isnull().sum().sum()}")


In [ ]:
# Export cleaned dataset
output_path = '../data/benin-malanville_clean.csv'
df_clean.to_csv(output_path, index=False)
print(f"✓ Cleaned dataset exported to: {output_path}")
print(f"✓ File size: {len(df_clean)} rows × {len(df_clean.columns)} columns")
